## Step 01: Import pyspark and initialize Spark

In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.mongodb.spark:mongo-spark-connector_2.11:2.4.0 pyspark-shell'
# create entry points to spark
from pyspark import SparkContext, SparkConf # Spark
from pyspark.sql import SparkSession # Spark SQL
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark_conf = SparkConf()
# setup application name
spark_conf.setAppName("Crime Analyzer")
# setup working processors
spark_conf.setMaster('local[*]')

# We add this line to avoid an error : "Cannot run multiple SparkContexts at once". 
# If there is an existing spark context, we will reuse it instead of creating a new context.
sc = SparkContext.getOrCreate()

# If there is no existing spark context, we now create a new context
if (sc is None):
    sc = SparkContext(conf = spark_conf)
                                    
spark = SparkSession\
        .builder\
        .appName("Crime Analyzer")\
        .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.11:2.4.0")\
        .config("spark.mongodb.input.uri","mongodb://127.0.0.1/crime.records")\
        .config("spark.mongodb.output.uri", "mongodb://127.0.0.1/crime.records")\
        .getOrCreate()

## Step 02: Create Dataframe

In [2]:
# create spark dataframe from csv file
crime = spark.read.format("csv").option("header","true").load("Crime_Statistics_SA_2010_present.csv")
no_of_records = crime.count()
print("There are ", no_of_records, " records in the data, including header" )
#crime.show()

There are  727408  records in the data, including header


## Step03: Write to Database

In [3]:
# write dataframe to MongoDB
crime.write.format("com.mongodb.spark.sql.DefaultSource").mode("overwrite").save()

## Step04: Read from Database

In [4]:
crime_records = spark.read.format("com.mongodb.spark.sql.DefaultSource").load()
crime_records.show(5)

+-------------+---------------------------+---------------------------+---------------------------+-------------------+-------------+-----------------+--------------------+
|Offence Count|Offence Level 1 Description|Offence Level 2 Description|Offence Level 3 Description|Postcode - Incident|Reported Date|Suburb - Incident|                 _id|
+-------------+---------------------------+---------------------------+---------------------------+-------------------+-------------+-----------------+--------------------+
|            2|       OFFENCES AGAINST ...|       FRAUD DECEPTION A...|       Obtain benefit by...|               5000|    1/07/2010|         ADELAIDE|[5d637f3556389f0c...|
|            2|       OFFENCES AGAINST ...|       PROPERTY DAMAGE A...|       Other property da...|               5000|    1/07/2010|         ADELAIDE|[5d637f3556389f0c...|
|            1|       OFFENCES AGAINST ...|       SERIOUS CRIMINAL ...|        SCT - Non Residence|               5000|    1/07/2010|  

## Step05: Calculate Statistics

In [5]:
#changedTyped = crime_records.select(col("Offence Count").cast('integer'))
#offence_count = changedTyped.groupby().sum('Offence Count')
#offence_count.show()
#offence_mean = changedTyped.groupby().mean('Offence Count')
#offence_mean.show()

# calculate statistics of the 'Offence Count' column
df_stats = crime_records.select(sum(col('Offence Count')).alias('sum')
                               ,mean(col('Offence Count')).alias('mean')
                               ,stddev(col('Offence Count')).alias('std')
                               ,min(col('Offence Count')).alias('min')
                               ,max(col('Offence Count')).alias('max')
                              ).collect()
print('Sum of offence count is', df_stats[0]['sum'])
print('Mean of offence count is', df_stats[0]['mean'])
print('Standard deviation of offence count is', df_stats[0]['std'])
print('Minimum of offence count is', df_stats[0]['min'])
print('Maximum of offence count is', df_stats[0]['max'])
print('--------------------------------------------')

# calculate statistics of the 'Reported Date' column, 
# and since there is no meaning for caculating the mean,sum and standard deviation of dates,
# only the earliest and latest date are collected as follow
df_stats = crime_records.select(
                                min(col('Reported Date')).alias('min')
                               ,max(col('Reported Date')).alias('max')
                              ).collect()
print('Minimum/Earliest of reported date is', df_stats[0]['min'])
print('Maximum/Latest of reported date is', df_stats[0]['max'])


Sum of offence count is 852170.0
Mean of offence count is 1.1715174585892079
Standard deviation of offence count is 0.5787050930378098
Minimum of offence count is 1
Maximum of offence count is 9
--------------------------------------------
Minimum/Earliest of reported date is 1/01/2011
Maximum/Latest of reported date is 9/12/2018


## Step06: Change the data type of a column

In [6]:
# import datetime module
from datetime import datetime
# udf function that convert string to date type in certain format y will lead to unconverted data!
func = udf(lambda x: datetime.strptime(x, '%d/%m/%Y'), DateType())
# created new dataframe with udf function
crime_modified = crime_records.withColumn('Reported Date', func(col('Reported Date')))
# display the first 5 row of Reported Date column
crime_modified.select('Reported Date').show(5)
# string = '2/01/2018'
# def func(string):
#     return datetime.strptime(string, '%d/%m/%Y')
# print(type(func(string)))

+-------------+
|Reported Date|
+-------------+
|   2010-07-01|
|   2010-07-01|
|   2010-07-01|
|   2010-07-01|
|   2010-07-01|
+-------------+
only showing top 5 rows



## Step07: Preliminary data analysis

In [55]:
df_result = crime_modified.select('Offence Level 2 Description').distinct()
crime_modified.where(crime_modified['Offence Level 2 Description'].isNull()).count()
print("There are ", df_result.count() - 1, " level 2 offences")
df_result.show()
# crime_modified.filter(crime_modified['Offence Level 2 Description']==None).collect()
# df = crime_modified.filter(crime_modified["Offence Level 2 Description"] == None)
# df.show(5)
# crime_modified.where(crime_modified['Offence Level 2 Description'].isNull()).collect()

There are  9  level 2 offences
+---------------------------+
|Offence Level 2 Description|
+---------------------------+
|       ROBBERY AND RELAT...|
|       PROPERTY DAMAGE A...|
|       SEXUAL ASSAULT AN...|
|                       null|
|       HOMICIDE AND RELA...|
|       ACTS INTENDED TO ...|
|       SERIOUS CRIMINAL ...|
|       OTHER OFFENCES AG...|
|       FRAUD DECEPTION A...|
|       THEFT AND RELATED...|
+---------------------------+



In [31]:
# Find the number of offences against person
count_person = crime_modified.where(crime_modified['Offence Level 1 Description']=="OFFENCES AGAINST THE PERSON").select(sum(col('Offence Count')).alias("sum")).collect()
print("The number of offences against person is", count_person[0]['sum'])

The number of offences against person is 175644.0


In [19]:
# Find serious criminal tresspasses with more than 1 offence count
count_tresspass = crime_modified.where((crime_modified['Offence Level 2 Description']=="SERIOUS CRIMINAL TRESPASS") & 
                                       (crime_modified['Offence Count'] > 1 )).count()
print("The number of serious criminal trespass with more than 1 offence count is", count_tresspass)

The number of serious criminal trespass with more than 1 offence count is 8579


In [64]:
# The percentage of offences against property
# I take each offence count as a single criminal, not sure if its correct
count_property = crime_modified.where(crime_modified['Offence Level 1 Description']=="OFFENCES AGAINST PROPERTY").select(sum(col('Offence Count')).alias("sum")).collect()
property_no = count_property[0]['sum']
count_total = crime_modified.select(sum(col('Offence Count')).alias("sum")).collect()
total_no = count_total[0]['sum']
print(property_no)
print(total_no)
percentage = property_no / total_no
print("The percentage of offences against property is", percentage * 100, "%")

676526.0
852170.0
The percentage of offences against property is 79.3886196416208 %


## Step08: Visualization

In [ ]:
# import matplotlib
import matplotlib.pyplot as plt
